### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
data=pd.read_csv(r'C:\aplica\E2E-Binary-Class-ANN\data\Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(
        optimizer='adam',
        loss="binary_crossentropy",
        metrics=['accuracy'])

    return model

In [7]:
## Create a Keras classifier
model=KerasClassifier(
    layers=1,
    neurons=32,
    build_fn=create_model,
    verbose=1
)

In [8]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [9]:
# Perform grid search
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    n_jobs=-1,
    cv=3,
    verbose=1
)

grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50


c:\aplica\E2E-Binary-Class-ANN\.venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\aplica\E2E-Binary-Class-ANN\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 0.8002 - loss: 0.5396
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.8127 - loss: 0.4358
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - accuracy: 0.8181 - loss: 0.4260
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - accuracy: 0.8198 - loss: 0.4232
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - accuracy: 0.8275 - loss: 0.4064
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.8262 - loss: 0.4094
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.8385 - loss: 0.3859
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.8416 - loss: 0.3835
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.8464 - loss: 0.3744
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.8478 - loss: 0.3683
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.8572 - loss: 0.3622
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [10]:
# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.857250 using {'epochs': 50, 'layers': 1, 'neurons': 16}
